In [20]:
import sys
sys.path.append('/workspaces/match_football_prediction/utils')

In [21]:
from data_processing import *

In [22]:
# Đọc file dữ liệu mùa giải hiện tại và chuyển vào dataframe
df_saved_name = '/workspaces/match_football_prediction/dulieuhuanluyen/EPL23-24.csv'
df_ml = pd.read_csv(df_saved_name)  # Đọc file CSV vào dataframe df_ml
# Điền giá trị 0 vào các ô bị thiếu
df_ml = df_ml.fillna(0)  # Điền giá trị 0 vào các ô trống trong dataframe df_ml

# Chọn các cột mong muốn
desired_columns = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
data = df_ml.loc[:, desired_columns]  # Tạo dataframe data chỉ chứa các cột mong muốn
data

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,Burnley,Man City,0,3,A,6,17,1,8,11,8,6,5,0,0,1,0
1,Arsenal,Nott'm Forest,2,1,H,15,6,7,2,12,12,8,3,2,2,0,0
2,Bournemouth,West Ham,1,1,D,14,16,5,3,9,14,10,4,1,4,0,0
3,Brighton,Luton,4,1,H,27,9,12,3,11,12,6,7,2,2,0,0
4,Everton,Fulham,0,1,A,19,9,9,2,12,6,10,4,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Crystal Palace,Aston Villa,5,0,H,15,8,9,2,10,8,2,4,1,4,0,0
376,Liverpool,Wolves,2,0,H,36,4,14,3,14,11,10,2,1,1,0,1
377,Luton,Fulham,2,4,A,15,15,6,7,15,20,4,4,5,4,0,0
378,Man City,West Ham,3,1,H,28,3,12,2,3,12,11,2,0,1,0,0


In [23]:
# Đọc file dữ liệu mùa giải trước và chuyển vào dataframe
last_season = '/workspaces/match_football_prediction/dulieuhuanluyen/EPL22-23.csv'
last_season = pd.read_csv(last_season)  # Đọc file CSV vào dataframe last_season

# Điền giá trị 0 vào các ô bị thiếu
last_season = last_season.fillna(0)  # Điền giá trị 0 vào các ô trống trong dataframe last_season

# Tính tỷ lệ thắng trên sân nhà của mỗi đội từ cột 'FTR' (Full Time Result)
home_win = last_season.groupby('HomeTeam')['FTR'].apply(lambda x: (x == 'H').mean())
# Tính tỷ lệ hòa trên sân nhà của mỗi đội từ cột 'FTR'
home_draw = last_season.groupby('HomeTeam')['FTR'].apply(lambda x: (x == 'D').mean())
# Tính tỷ lệ thắng trên sân khách của mỗi đội từ cột 'FTR'
away_win = last_season.groupby('AwayTeam')['FTR'].apply(lambda x: (x == 'A').mean())
# Tính tỷ lệ hòa trên sân khách của mỗi đội từ cột 'FTR'
away_draw = last_season.groupby('AwayTeam')['FTR'].apply(lambda x: (x == 'D').mean())

# Tạo DataFrame mới từ các Series trên
last_season_stats = pd.DataFrame({
    'LSHW': home_win,  # LSHW: Tỷ lệ thắng trên sân nhà mùa trước
    'LSHD': home_draw, # LSHD: Tỷ lệ hòa trên sân nhà mùa trước
    'LSAW': away_win,  # LSAW: Tỷ lệ thắng trên sân khách mùa trước
    'LSAD': away_draw  # LSAD: Tỷ lệ hòa trên sân khách mùa trước
}).reset_index()

# Đổi tên cột 'index' thành 'Team' để dễ dàng merge
last_season_stats.rename(columns={'index': 'Team'}, inplace=True)

last_season_stats

,Team,LSHW,LSHD,LSAW,LSAD
0,Arsenal,0.736842,0.157895,0.631579,0.157895
1,Aston Villa,0.631579,0.105263,0.315789,0.263158
2,Bournemouth,0.315789,0.210526,0.263158,0.105263
3,Brentford,0.526316,0.368421,0.263158,0.368421
4,Brighton,0.526316,0.210526,0.421053,0.210526
5,Chelsea,0.315789,0.368421,0.263158,0.210526
6,Crystal Palace,0.368421,0.368421,0.210526,0.263158
7,Everton,0.315789,0.157895,0.105263,0.473684
8,Fulham,0.421053,0.263158,0.368421,0.105263
9,Leeds,0.263158,0.368421,0.105263,0.157895


In [24]:
teams = data['HomeTeam'].unique()
special_teams_map = {
    'Burnley': 'Leicester',
    'Sheffield United': 'Leeds',
    'Luton': 'Southampton'
}
home_l5m = dudoan_home_2_l5m(data,teams)
home_l5m=home_l5m[['Team','FTHG', 'HC', 'HF', 'HR', 'HS', 'HST', 'HY']]
home_l5m = home_l5m.merge(last_season_stats[['Team','LSHW', 'LSHD']],
                          left_on='Team', right_on='Team', how='left')
# Thay thế thông số của các đội đặc biệt bằng thông số của đội tương ứng
for team, replace_with in special_teams_map.items():
        replace_stats = last_season_stats.loc[last_season_stats['Team'] == replace_with, ['LSHW', 'LSHD']].iloc[0]
        home_l5m.loc[home_l5m['Team'] == team, ['LSHW', 'LSHD']] = replace_stats[['LSHW', 'LSHD']].values

away_l5m = dudoan_away_2_l5m(data,teams)
away_l5m=away_l5m[['Team','FTAG', 'AC', 'AF', 'AR', 'AS', 'AST', 'AY']]
away_l5m = away_l5m.merge(last_season_stats[['Team','LSAW', 'LSAD']],
                          left_on='Team', right_on='Team', how='left')
for team, replace_with in special_teams_map.items():
        replace_stats = last_season_stats.loc[last_season_stats['Team'] == replace_with, ['LSAW', 'LSAD']].iloc[0]   
        away_l5m.loc[away_l5m['Team'] == team, ['LSAW', 'LSAD']] = replace_stats[['LSAW', 'LSAD']].values


In [33]:
from sklearn.preprocessing import StandardScaler

new_columns = {
   'Team': 'HomeTeam',
   'FTHG': 'FTHG_home',
   'HC': 'HC_home',
   'HF': 'HF_home',
   'HR': 'HR_home',
   'HS': 'HS_home',
   'HST': 'HST_home',
   'HY': 'HY_home',
   'LSHW': 'LSHW_home',
   'LSHD': 'LSHD_home'
}
home_l5m = home_l5m.rename(columns=new_columns)

new_columns = {
   'Team': 'AwayTeam',
   'FTAG': 'FTAG_away',
   'AC': 'AC_away',
   'AF': 'AF_away',
   'AR': 'AR_away',
   'AS': 'AS_away',
   'AST': 'AST_away',
   'AY': 'AY_away',
   'LSAW': 'LSAW_away',
   'LSAD': 'LSAD_away'
}
away_l5m = away_l5m.rename(columns=new_columns)

scaler = StandardScaler()

# Đối với dữ liệu của đội nhà
cols_to_scale_home = [col for col in home_l5m.columns if col != 'HomeTeam']
home_l5m[cols_to_scale_home] = scaler.fit_transform(home_l5m[cols_to_scale_home])

# Đối với dữ liệu của đội khách
cols_to_scale_away = [col for col in away_l5m.columns if col != 'AwayTeam']
away_l5m[cols_to_scale_away] = scaler.fit_transform(away_l5m[cols_to_scale_away])

In [34]:
home_l5m

,HomeTeam,FTHG_home,HC_home,HF_home,HR_home,HS_home,HST_home,HY_home,LSHW_home,LSHD_home
0,Burnley,-0.759499,-0.983809,0.744707,-0.420084,-0.115274,-0.687581,-0.755610,-1.124104,-0.186352
1,Arsenal,0.524162,-0.841228,0.644071,-0.420084,1.286097,0.257187,0.988105,1.284691,-0.718787
2,Bournemouth,-0.117669,0.727163,1.650431,-0.420084,-0.476918,-0.477633,0.988105,-0.856460,-0.186352
3,Brighton,-1.615274,-0.128323,0.644071,-0.420084,-0.883768,-1.212452,-0.174371,0.214115,-0.186352
4,Everton,-0.545556,-1.268971,-0.563562,-0.420084,-0.567329,-1.002504,-0.464991,-0.856460,-0.718787
5,Sheffield United,-0.545556,-0.556066,-0.161018,2.380476,-0.928973,-0.267684,0.697486,-1.124104,1.410952
6,Newcastle,1.165992,1.012325,-0.362290,2.380476,0.788836,0.467135,-1.336848,0.481759,0.878518
7,Brentford,-0.973443,-0.413485,-1.167378,-0.420084,-0.838562,-1.002504,0.697486,0.214115,1.410952
8,Chelsea,2.021766,0.442001,-1.268014,-0.420084,0.834042,1.621852,-0.464991,-0.856460,1.410952
9,Man United,0.096275,0.442001,-0.966106,-0.420084,0.517603,0.887032,0.116248,1.552334,-0.718787


In [35]:
away_l5m

,AwayTeam,FTAG_away,AC_away,AF_away,AR_away,AS_away,AST_away,AY_away,LSAW_away,LSAD_away
0,Burnley,0.214902,-0.729282,-0.269953,2.883223,0.083772,0.677578,0.326164,-0.504902,-0.718787
1,Arsenal,0.521905,0.323290,0.854850,-0.611593,0.462096,0.543404,-0.326164,2.280763,-0.718787
2,Bournemouth,-1.013109,0.022555,1.079811,1.135815,0.570189,-0.261639,0.000000,-0.156694,-1.251222
3,Brighton,-1.320112,-0.578915,-0.494913,-0.611593,1.056606,0.006709,0.652328,0.887931,-0.186352
4,Everton,-1.013109,0.473657,1.417252,-0.611593,-1.051202,-1.200856,1.630820,-1.201318,2.475822
5,Sheffield United,-1.013109,-0.278180,-0.719874,-0.611593,-0.402645,-0.932508,-1.630820,-1.201318,-0.718787
6,Newcastle,1.135910,0.172923,-0.269953,-0.611593,0.732328,1.482621,1.304656,0.887931,1.943387
7,Brentford,1.135910,0.924760,0.517409,1.135815,-0.132414,0.543404,0.652328,-0.156694,1.410952
8,Chelsea,0.521905,0.022555,0.067488,1.135815,-0.132414,-0.261639,1.304656,-0.156694,-0.186352
9,Man United,0.214902,-1.030017,0.404929,-0.611593,-0.294553,-0.529987,-1.304656,0.887931,-0.718787


In [37]:
import joblib
import pandas as pd

# Load model từ file
loaded_model = joblib.load(r'/workspaces/match_football_prediction/train_models/EPL_train/best_random_forest_L5M.joblib')

home_l5m_copy=home_l5m.copy()
away_l5m_copy=away_l5m.copy()

In [38]:
home_l5m_copy.to_csv('dudoan_home.csv')
away_l5m_copy.to_csv('dudoan_away.csv')

In [50]:
import joblib
import pandas as pd

# Load model từ file
loaded_model = joblib.load(r'/workspaces/match_football_prediction/train_models/EPL_train/best_random_forest_L5M.joblib')
# Mã hóa tên đội thành số
home_l5m['HomeTeam_code'] = home_l5m['HomeTeam'].astype('category').cat.codes 
away_l5m['AwayTeam_code'] = away_l5m['AwayTeam'].astype('category').cat.codes

# Tạo dictionary để map tên đội với mã số
team_dict = dict(zip(home_l5m['HomeTeam'], home_l5m['HomeTeam_code']))
# Nhập tên đội nhà và đội khách
home_team = input("Nhập tên đội nhà: ")
away_team = input("Nhập tên đội khách: ")

# Lấy mã số của đội nhà và đội khách
home_team_code = team_dict[home_team]
away_team_code = team_dict[away_team]

# Lấy thông tin đội nhà và đội khách dựa trên mã số
home_info = home_l5m[home_l5m['HomeTeam_code'] == home_team_code].iloc[0]
away_info = away_l5m[away_l5m['AwayTeam_code'] == away_team_code].iloc[0]

# Tạo dataframe đầu vào cho model
input_data = pd.DataFrame({
    'HomeTeam': [home_info['HomeTeam_code']],
    'AwayTeam': [away_info['AwayTeam_code']],
    'AC_away': [away_info['AC_away']],
    'AF_away': [away_info['AF_away']],
    'AR_away': [away_info['AR_away']],
    'AS_away': [away_info['AS_away']],
    'AST_away': [away_info['AST_away']],
    'AY_away': [away_info['AY_away']],
    'FTAG_away': [away_info['FTAG_away']],
    'FTHG_home': [home_info['FTHG_home']],
    'HC_home': [home_info['HC_home']],
    'HF_home': [home_info['HF_home']],
    'HR_home': [home_info['HR_home']],
    'HS_home': [home_info['HS_home']],
    'HST_home': [home_info['HST_home']],
    'HY_home': [home_info['HY_home']],
    'LSHW_home': [home_info['LSHW_home']],
    'LSHD_home': [home_info['LSHD_home']],
    'LSAW_away': [away_info['LSAW_away']],
    'LSAD_away': [away_info['LSAD_away']]
})
desired_order = [
    'HomeTeam', 'AwayTeam','LSHW_home','LSAW_away','LSHD_home','LSAD_away','FTHG_home','FTAG_away','HC_home','AC_away','HF_home',
    'AF_away','HR_home','AR_away', 'HS_home','AS_away', 'HST_home', 'AST_away','HY_home','AY_away'
]

# Sắp xếp lại các cột theo thứ tự mong muốn
input_data = input_data.reindex(columns=desired_order)

# Dự đoán kết quả
prediction = loaded_model.predict(input_data)
# In kết quả
result = ""
if prediction[0] == 2:
    result = "thắng"
elif prediction[0] == 0:
    result = "thua"
else:
    result = "hòa"
print("Kết quả dự đoán trận đấu giữa", home_team, "và", away_team, "là:", result)

Kết quả dự đoán trận đấu giữa Man City và Tottenham là: thắng
